# Wrangle and Analyze Data of a Twitter Account


## Table of Contents
- [1. Introduction](#intro)
- [2. Gather data](#gather)
- [3. Assess data](#assess)
- [4. Clean data](#clean)
- [5. Store](#store)


<a id='gather'></a>
## 1. Introduction

This project is an analysis of correlation between the Covid-19 cases and the political environment of different countries. Goal is to find answers or at least indicators to questions like: 
- Did the countries which had more success in containing the amount of Covid-19 cases something in common? 
- Is there a correlation in  Gross domestic product, Human Development Index or political ideology with the amount of Covid-19 cases of the country.

Main goal of this project is to generate a comprehensive exploratory and explanatory data analysis of the gathered data. The data analysis process is distributed over three ipynb-files: gather_clean_Covid19.ipynb, exploration_Covid19.ipynb and slide_deck_Covid19.ipynb.

Firstly, as part of gather_clean_Covid19.ipynb data is gathered from different sources: The Covid-19 data of this project is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19) and additional data about countries is retrieved via the wikipedia API. Secondly, the data from the different sources is visually and programmatically assessed to be cleaned.
The exploratory and explanatory data analysis of the gathered data is performed in exploration_Covid19.ipynb. Finally the findings are presented in slide_deck_Covid19.ipynb.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import date
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os # to work with local directory
import re
import wptools
import json # to create json file from python dictionary
import time # for timer 
sns.set()

<a id='intro'></a>
## 2. Gather data

####  Data is gathered from three different sources of data as described in steps below:

1. Fatality, confirmed cases, recovered cases and data by country is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19).
2. Additional data is retrieved via the wptools API from different wikipedia articles.

### a. Read data from programmatically download csv-file

In [2]:
# Gather data from John Hopkins GitHub 
df_JHU_Fatality = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_JHU_Confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_JHU_Recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df_JHU_Countries = pd.read_csv('https://raw.githubusercontent.com/RRighart/covid-19/master/countries.csv')

In [3]:
df_OWID_Covid = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
df_OWID_Testing = pd.read_csv('https://covid.ourworldindata.org/data/testing/covid-testing-latest-data-source-details.csv')
df_OWID_Countries = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

### b. Read data from local datasets

Data downloaded manually from different databases, [European statistical database](https://ec.europa.eu/eurostat/data/database), [Wikipedia table on intensive care units](https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds) and [United Nations database](https://data.un.org):

In [4]:
df_ESTAT_census = pd.read_csv('inputData/Eurostat_HouseholdSize_2001.csv')
df_WIKI_ICU = pd.read_csv('inputData/Wikipedia_ICU.csv')
df_UN_births = pd.read_csv('inputData/UNdata_birthsByMonth.csv')
df_UN_deaths = pd.read_csv('inputData/UNdata_deathsByMonth.csv')

### c. Query additional information for countries via wikipedia API

Additional Information
- Leader Gender
- Ideology of Leading Party
- Amount of Intensive Care Beds
- Gross domestic product per capita
- Human Development Index

In [5]:
# Query for every tweet id in enhanced twitter archive and save tweet-information in json-format to 'tweet_json.txt'
'''             
country_jsons = {}
county_id_errors = []
start = time.time()
count = 0


with open('country_json.txt', 'w') as outfile:
    
    for country in df_JHU_Countries['Country/Region']:
        count +=1
        try:
            # Query API for data of wikipedia article
            article = wptools.page(country).get_parse()
            infobox = article.data['infobox']
            # Measure elapsed time
            mid_s = time.time()
            # Print id and time elapsed
            print(str(count) + str(mid_s - start) )
            # Write json of tweet to 'tweet_json.txt'
            json.dump(infobox, outfile)
            # New line
            outfile.write("\n")

        # Not best practice to catch all exceptions but fine for this short script
        except Exception as error:
            mid_f = time.time()
            print(str(count) + str(mid_f - start) + str(error))
            # Gather ids of id's without status
            tweet_id_errors.append([count, str(tweet_id)])
            
    end = time.time()
    print(end - start)
    
    '''

'             \ncountry_jsons = {}\ncounty_id_errors = []\nstart = time.time()\ncount = 0\n\n\nwith open(\'country_json.txt\', \'w\') as outfile:\n    \n    for country in df_JHU_Countries[\'Country/Region\']:\n        count +=1\n        try:\n            # Query API for data of wikipedia article\n            article = wptools.page(country).get_parse()\n            infobox = article.data[\'infobox\']\n            # Measure elapsed time\n            mid_s = time.time()\n            # Print id and time elapsed\n            print(str(count) + str(mid_s - start) )\n            # Write json of tweet to \'tweet_json.txt\'\n            json.dump(infobox, outfile)\n            # New line\n            outfile.write("\n")\n\n        # Not best practice to catch all exceptions but fine for this short script\n        except Exception as error:\n            mid_f = time.time()\n            print(str(count) + str(mid_f - start) + str(error))\n            # Gather ids of id\'s without status\n       

In [6]:
'''
so = wptools.page('Germany').get_parse()
infobox = so.data['infobox']
print(infobox)
'''

"\nso = wptools.page('Germany').get_parse()\ninfobox = so.data['infobox']\nprint(infobox)\n"

<a id='assess'></a>
## 3. Assess data

After gathering each of the above pieces of data, they are assessed visually and programmatically for quality and tidiness issues. Requirements to be met:

- Quality requirements:
    - Completeness: All necessary records in dataframes, no specific rows, columns or cells missing.
    - Validity: No records available, that do not conform schema.
    - Accuracy: No wrong data, that is valid.
    - Consistency: No data, that is valid and accurate, but referred to in multiple correct ways.
- Tidiniss requirements (as defined by Hadley Wickham):
    - each variable is a column
    - each observation is a row
    - each type of observational unit is a table.

### a. Visual assessment

In [7]:
# Check layout of df_JHU_Countries vsiually
df_JHU_Countries.sample(n=5)

,Unnamed: 0,Country/Region,inhabitants,area
26,26,India,1380004385,2973190
21,21,Sweden,10099265,410340
6,6,Italy,60496082,301338
8,8,United Kingdom,67803450,242495
22,22,Denmark,5792202,42430


In [8]:
# Check layout of df_JHU_Fatality vsiually
df_JHU_Fatality.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
48,NaN,Chile,-35.6751,-71.5430,0,0,0,0,0,0,...,335,346,368,394,421,450,478,509,544,589
174,NaN,North Macedonia,41.6086,21.7453,0,0,0,0,0,0,...,92,95,95,97,98,101,104,106,110,111
173,NaN,Nigeria,9.0820,8.6753,0,0,0,0,0,0,...,158,164,167,171,176,182,191,192,200,211
39,New Brunswick,Canada,46.5653,-66.4619,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,Chongqing,China,30.0572,107.8740,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


In [9]:
# Check layout of df_JHU_Confirmed vsiually
df_JHU_Confirmed.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
247,NaN,Burma,21.916200,95.956000,0,0,0,0,0,0,...,180,181,181,182,182,184,191,193,199,199
234,NaN,Mozambique,-18.665695,35.529562,0,0,0,0,0,0,...,104,104,115,119,129,137,145,146,156,162
209,NaN,Thailand,15.000000,101.000000,2,3,5,7,8,8,...,3017,3017,3018,3025,3025,3028,3031,3033,3034,3037
137,NaN,Italy,43.000000,12.000000,0,0,0,0,0,0,...,221216,222104,223096,223885,224760,225435,225886,226699,227364,228006
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,6563,6879,7134,7479,7805,8068,8371,8809,9283,9931


In [10]:
# Check layout of df_JHU_Recovered vsiually
df_JHU_Recovered.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
153,NaN,Mauritius,-20.2000,57.5000,0,0,0,0,0,0,...,322,322,322,322,322,322,322,322,322,322
132,NaN,Jamaica,18.1096,-77.2975,0,0,0,0,0,0,...,100,113,118,121,121,127,131,145,171,181
194,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,3851,4809,5973,7248,8342,9340,9835,10365,11207,12117
29,NaN,Brazil,-14.2350,-51.9253,0,0,0,0,0,0,...,72597,78424,79479,84970,89672,94122,100459,106794,116683,125960
59,Liaoning,China,41.2956,122.6085,0,0,0,0,0,0,...,144,144,144,144,144,144,144,144,144,144


In [11]:
# Check layout of df_OWID_Covid vsiually
df_OWID_Covid.sample(n=5)

# df_OWID_Covidchange 'location' to 'country'
# df_OWID_Covid create df_OWID_Countries with 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'
# df_OWID_Covid merge it to df_country

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
18877,OWID_WRL,World,2020-01-14,60,0,1,0,0.008,0.000,0.0,...,8.696,5.355,15469.207,10.0,233.070,8.51,6.434,34.635,60.130,2.705
9557,KAZ,Kazakhstan,2020-03-23,60,4,0,0,3.195,0.213,0.0,...,6.991,4.625,24055.588,0.1,466.792,7.11,7.000,43.100,98.999,6.700
17666,UGA,Uganda,2020-04-10,53,0,0,0,1.159,0.000,0.0,...,2.168,1.308,1697.707,41.6,213.333,2.50,3.400,16.700,21.222,0.500
276,AIA,Anguilla,2020-04-15,3,0,0,0,199.973,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7204,GRD,Grenada,2020-03-23,1,1,0,0,8.887,8.887,0.0,...,7.304,5.021,13593.877,NaN,243.964,10.71,NaN,NaN,NaN,3.700


In [12]:
# Check layout of df_OWID_Testing vsiually
df_OWID_Testing.sample(n=5)

# df_OWID_Testing drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
# df_OWID_Testing Either cut per regex country name from 'Entity' and rename country or join country name from other df

,ISO code,Entity,Date,Source URL,Source label,Notes,Number of observations,Cumulative total,Cumulative total per thousand,Daily change in cumulative total,Daily change in cumulative total per thousand,7-day smoothed daily change,7-day smoothed daily change per thousand,General source label,General source URL,Short description,Detailed description
34,IRN,Iran - tests performed,2020-05-19,http://web.archive.org/web/20200519130712/http...,Government of Iran,NaN,38,716176,8.527,14536.0,0.173,14386,0.171,Government of Iran,http://irangov.ir/,The number of tests performed.,The Government of Iran provides daily press re...
30,ISL,Iceland - samples tested,2020-05-19,https://www.covid.is/tolulegar-upplysingar,Office of the Director of Public Health,NaN,83,57587,168.753,441.0,1.292,365,1.070,Office of the Director of Public Health,https://www.covid.is/tolulegar-upplysingar,"The number of samples (""Fjöldi sýna"").",The Icelandic Office of Public Health publishe...
46,MYS,Malaysia - people tested,2020-05-20,https://web.archive.org/web/20200520182204/htt...,Ministry of Health Malaysia,NaN,70,462257,14.282,NaN,NaN,18425,0.569,Ministry of Health Malaysia,http://www.moh.gov.my/index.php/pages/view/201...,"The number of cases tested, including pending.",A time series has not yet been released. The f...
3,BHR,Bahrain - units unclear,2020-05-20,https://web.archive.org/web/20200520201058/htt...,Ministry of Health,NaN,61,255633,150.232,7428.0,4.365,7152,4.203,Bahrain Ministry of Health,https://www.moh.gov.bh/COVID19,The source reports the 'number of assessments'...,The Bahrainian Ministry of Health publishes fr...
56,PAK,Pakistan - tests performed,2020-05-20,https://web.archive.org/web/20200520182803/htt...,Government of Pakistan,NaN,70,414254,1.875,13962.0,0.063,13794,0.062,Government of Pakistan,http://www.covid.gov.pk/,The number of tests performed.,The government of Pakistan publishes a [dashbo...


In [13]:
# Check layout of df_OWID_Countries vsiually
df_OWID_Countries.sample(n=5)

# df_OWID_Countries convert datatype population to integer
# df_OWID_Countries drop 'countriesAndTerritories', 'population_year'

,countriesAndTerritories,location,continent,population_year,population
69,Finland,Finland,Europe,2020.0,5540718.0
93,Indonesia,Indonesia,Asia,2020.0,273523621.0
169,Seychelles,Seychelles,Africa,2020.0,98340.0
153,Philippines,Philippines,Asia,2020.0,109581085.0
187,Tajikistan,Tajikistan,Asia,2020.0,9537642.0


In [14]:
# Check layout of df_ESTAT_census vsiually
df_ESTAT_census.sample(n=5)

# df_ESTAT_census make columns from values in n_person
# df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'

,TIME,GEO,N_PERSON,Value
588,2012,Greece,1 person,20.4
96,2010,France,1 person,35.3
2419,2019,Netherlands,2 persons,:
1379,2015,Cyprus,6 persons or more,3.2
115,2010,Cyprus,2 persons,30.2


In [15]:
df_ESTAT_census.GEO.unique()

array(['European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)',
       'European Union - 27 countries (from 2020)',
       'European Union - 28 countries (2013-2020)',
       'European Union - 27 countries (2007-2013)',
       'Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015)',
       'Euro area - 19 countries  (from 2015)',
       'Euro area - 18 countries (2014)', 'Belgium', 'Bulgaria',
       'Czechia', 'Denmark',
       'Germany (until 1990 former territory of the FRG)', 'Estonia',
       'Ireland', 'Greece', 'Spain', 'France', 'Croatia', 'Italy',
       'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Malta',
       'Netherlands', 'Austria', 'Poland', 'Portugal', 'Romania',
       'Slovenia', 'Slovakia', 'Finland', 'Sweden', 'United Kingdom',
       'Iceland', 'Norway', 'Switzerland', 'Montenegro',
       'North Macedonia', 'Serbia', 'Turkey'], dtype=object)

In [16]:
# Check layout of df_WIKI_ICU vsiually
df_WIKI_ICU.sample(n=5)

,countryname,continent,hospital_beds_per_1000_people,occupancy,ICU-CCB_beds_per_1000_people,ventilators
40,Mexico,North America,1.38,74.0,1.2,2050.0
8,Lithuania,Europe,6.56,73.2,15.5,1000.0
42,Bangladesh,Asia,0.87,NaN,0.72,NaN
26,Iceland,Europe,3.06,NaN,9.1,NaN
4,Austria,Europe,7.37,73.8,21.8,2500.0


In [17]:
df_UN_births.sample(n=5)
# Drop columns 'Area', 'Record Type', 'Reliability', 'Value Footnotes', 'Source Year'
# change datatype of columns  'Value' to integer
# Merge df_UN_births and df_UN_deaths on Year


,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
9367,Suriname,2016,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2018.0,792.0,NaN
1255,Bermuda,2017,Total,July,Data tabulated by year of occurrence,"Final figure, complete",2019.0,50.0,8
3470,France,2015,Total,February,Data tabulated by year of occurrence,Provisional figure,2015.0,57500.0,NaN
5581,Lithuania,2015,Total,March,Data tabulated by year of occurrence,"Final figure, complete",2017.0,2543.0,NaN
8542,San Marino,2014,Total,November,Data tabulated by year of occurrence,"Final figure, complete",2016.0,17.0,NaN


In [18]:
df_UN_births.Area.value_counts()

Total    10373
Name: Area, dtype: int64

In [19]:
df_UN_deaths.sample(n=5)

,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
519,Australia,2014,Total,June,Data tabulated by year of occurrence,"Final figure, complete",2017.0,13083.0,NaN
6506,North Macedonia,2019,Total,September,Data tabulated by year of occurrence,Provisional figure,2019.0,1437.0,NaN
6063,Montenegro,2014,Total,Total,Data tabulated by year of occurrence,"Final figure, complete",2016.0,6014.0,NaN
1779,"China, Hong Kong SAR",2013,Total,July,Data tabulated by year of occurrence,"Final figure, complete",2015.0,3529.0,NaN
7722,Russian Federation,2013,Total,October,Data tabulated by year of occurrence,Provisional figure,2013.0,168671.0,1


### b. Programmatic assessment

In [20]:
# List of countries that are avaoilable in John Hopkins Dataset
df_JHU_Recovered['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Diamond Princess',
       'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Grenada', 'Greece', 'Guatemala', 'Guinea',
       'Guinea-Bissau', 'Guyana', 'Haiti', 'H

In [21]:
# List of countries that are avaoilable in John Hopkins Dataset
df_OWID_Covid['location'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Western Sahara', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'Falkland Islands', 'France',

In [22]:
# Available variables in dataset
list(df_OWID_Covid)

['iso_code',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'new_tests_smoothed',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed_per_thousand',
 'tests_units',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [23]:
df_OWID_Covid.query('location == "Germany" and date == "2020-05-13"')

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
4502,DEU,Germany,2020-05-13,171306,798,7634,101,2044.616,9.524,91.115,...,21.453,15.957,45229.245,NaN,156.139,8.31,28.2,33.1,NaN,8.0


In [24]:
df_Check = df_JHU_Confirmed.copy()
df_Check.rename(columns={'Country/Region': 'country'}, inplace=True)
df_Check.query('country == "Germany"')

,Province/State,country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,...,173171,174098,174478,175233,175752,176369,176551,177778,178473,179021


In [25]:
# Check for countries which are referred to by different names in different dataframes
c_df_JHU_Fatality =  df_JHU_Fatality['Country/Region'].unique()
c_df_JHU_Confirmed = df_JHU_Confirmed['Country/Region'].unique()
c_df_JHU_Recovered = df_JHU_Recovered['Country/Region'].unique()
c_df_JHU_Countries = df_JHU_Countries['Country/Region'].unique()
c_df_OWID_Covid = df_OWID_Covid['location'].unique()
c_df_OWID_Testing = df_OWID_Testing['Entity'].unique()
c_df_OWID_Countries = df_OWID_Countries['location'].unique()
c_df_WIKI_ICU = df_WIKI_ICU['countryname'].unique()
c_df_UN_births = df_UN_births['Country or Area'].unique()
c_df_UN_deaths = df_UN_deaths['Country or Area'].unique()

all_country_names = list(c_df_JHU_Fatality) + list(c_df_JHU_Confirmed) + list(c_df_JHU_Recovered) + list(c_df_JHU_Countries) + list(c_df_OWID_Covid) + list(c_df_OWID_Testing) + list(c_df_OWID_Countries) + list(c_df_WIKI_ICU) + list(c_df_UN_births) + list(c_df_UN_deaths)

all_country_names = pd.Series(all_country_names).unique()

In [26]:
all_country_names

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

### Findings, which contradict requirements:

#### Quality Observations:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.
- Consistency: Data about Covid-19 cases differs slightly between John Hopkins and OWID, data which is available in both datasets will be kept only from John Hopkins.
- Consistency: Some countries are referred to with varying names, for example 'US' and 'United Stats'. Other names are not valid.

#### Tidiness Observations:
- The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.
- Column 'Country/Region' should only contain countries, therefore column name should by 'country', same for OWID data.
- Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'
- Data for countries, which are not of interested is not needed in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality', 'df_JHU_Countries'
- In the df_OWID_Covid dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.
- In the df_OWID_Testing dataframes there is data which is not of interest.
- Datatype of variable population in df_OWID_Countries is not integer and there are variables not of interest for this project 'countriesAndTerritories' and 'population_year'
- Tidiniss: Colum 'Value' contains more than one variable and observations are spread over multiple rows. Each variable should be a column, each observation a row.


<a id='clean'></a>
## 4. Clean data

In [27]:
# Create copies for cleaning process to preserve original dataframes
df_JHU_Fatality_clean = df_JHU_Fatality.copy()
df_JHU_Confirmed_clean = df_JHU_Confirmed.copy()
df_JHU_Recovered_clean = df_JHU_Recovered.copy()
df_JHU_Countries_clean = df_JHU_Countries.copy()
df_OWID_Covid_clean = df_OWID_Covid.copy()
df_OWID_Testing_clean = df_OWID_Testing.copy()
df_OWID_Countries_clean = df_OWID_Countries.copy()
df_ESTAT_census_clean = df_ESTAT_census.copy()
df_WIKI_ICU_clean = df_WIKI_ICU.copy()
df_UN_births_clean = df_UN_births.copy()
df_UN_deaths_clean = df_UN_deaths.copy()

### Issue 1:
#### Observe:
-  Tidiness: Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'

#### Define:
- Drop columns 'Province/State', 'Lat' and 'Long'

#### Code:

In [28]:
# Drop variables which are only necessary for retweets
df_JHU_Fatality_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Confirmed_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Recovered_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

#### Test:

In [29]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Fatality_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,127,132,136,153,168,169,173,178,187,193


In [30]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Confirmed_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676


In [31]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Recovered_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,610,648,691,745,745,778,801,850,930,938


### Issue 2:
#### Observe:
- Tidiness: Column 'Country/Region' should only contain countries, therefore column name should by 'Country'.

#### Define:
- Rename column 'Country/Region' to 'country'

#### Code:

In [32]:
# Rename coloumn inplace to identic primary key
df_JHU_Fatality_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Confirmed_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Recovered_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Countries_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_OWID_Covid_clean.rename(columns={'location': 'country'}, inplace=True)
df_OWID_Countries_clean.rename(columns={'location': 'country'}, inplace=True)
df_ESTAT_census_clean.rename(columns={'GEO': 'country'}, inplace=True)

df_OWID_Testing_clean.rename(columns={'Date': 'date'}, inplace=True)

#### Test:

In [33]:
assert df_JHU_Fatality_clean.country.any()

In [34]:
assert df_JHU_Confirmed_clean.country.any()

In [35]:
assert df_JHU_Recovered_clean.country.any()

In [36]:
assert df_JHU_Countries_clean.country.any()

In [37]:
assert df_OWID_Covid_clean.country.any()

In [38]:
assert df_OWID_Countries_clean.country.any()

### Issue 3:
#### Observe:
- Tidiness: In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.

#### Define
- Create new dataframe df_OWID_Countries with columns 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'.

#### Code:

In [39]:
list(df_OWID_Covid_clean)

['iso_code',
 'country',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'new_tests_smoothed',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed_per_thousand',
 'tests_units',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [40]:
df_OWID_Covid_clean = df_OWID_Covid_clean.copy()
df_OWID_Covid_clean.drop([
                         'total_cases',
                         'new_cases',
                         'total_deaths',
                         'new_deaths',
                         'total_cases_per_million',
                         'new_cases_per_million',
                         'total_deaths_per_million',
                         'new_deaths_per_million',
                         'total_tests',
                         'new_tests',
                         'total_tests_per_thousand',
                         'new_tests_per_thousand',
                         'tests_units',
                         'cvd_death_rate',
                         'handwashing_facilities',
                         'extreme_poverty'], axis=1, inplace=True)
df_OWID_Covid_clean = df_OWID_Covid_clean.drop_duplicates()


#### Test:

In [41]:
df_OWID_Covid_clean.country.value_counts().head(3)

Belgium        144
Switzerland    144
Denmark        144
Name: country, dtype: int64

In [42]:
list(df_OWID_Covid_clean)

['iso_code',
 'country',
 'date',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'hospital_beds_per_100k']

### Issue 4:
#### Observe:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.

#### Define: 
- Sum values of rows with same entry in column country by using groupby

#### Code:

In [43]:
# Groupby and sum
df_JHU_Fatality_clean = df_JHU_Fatality_clean.groupby(['country'], as_index=False).sum()
df_JHU_Confirmed_clean = df_JHU_Confirmed_clean.groupby(['country'], as_index=False).sum()
df_JHU_Recovered_clean = df_JHU_Recovered_clean.groupby(['country'], as_index=False).sum()

#### Test:

In [44]:
df_JHU_Fatality_clean.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
183    False
184    False
185    False
186    False
187    False
Length: 188, dtype: bool

### Issue 5:
#### Observe:
- Tidiness: The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.

#### Define:
- Melt date columns to one column 'date', transform date to type datetime and merge the three dataframes to ones dataframe 'df_covid' with sorted date values.

#### Code:

In [45]:
# Melt each dataframe so that results in columns: country,
df_JHU_Fatality_clean = pd.melt(df_JHU_Fatality_clean, id_vars = ['country'], var_name='date', value_name='fatal')
df_JHU_Confirmed_clean = pd.melt(df_JHU_Confirmed_clean, id_vars = ['country'], var_name='date', value_name='confirmed')
df_JHU_Recovered_clean = pd.melt(df_JHU_Recovered_clean, id_vars = ['country'], var_name='date', value_name='recovered')

In [46]:
# Convert new columns date to datetime
df_JHU_Fatality_clean.date=pd.to_datetime(df_JHU_Fatality_clean.date)
df_JHU_Confirmed_clean.date=pd.to_datetime(df_JHU_Confirmed_clean.date)
df_JHU_Recovered_clean.date=pd.to_datetime(df_JHU_Recovered_clean.date)

In [47]:
# Merge three covid dataframes to one
df_covid = pd.merge(df_JHU_Fatality_clean, df_JHU_Confirmed_clean, on=['country','date'])
df_covid = pd.merge(df_covid, df_JHU_Recovered_clean, on=['country','date'])

In [48]:
# Sort date values by date
df_covid = df_covid.sort_values(by='date', ascending=True)

#### Test:

In [49]:
list(df_covid)

['country', 'date', 'fatal', 'confirmed', 'recovered']

In [50]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22748 entries, 0 to 22747
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   country    22748 non-null  object        
 1   date       22748 non-null  datetime64[ns]
 2   fatal      22748 non-null  int64         
 3   confirmed  22748 non-null  int64         
 4   recovered  22748 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.0+ MB


In [51]:
df_covid

,country,date,fatal,confirmed,recovered
0,Afghanistan,2020-01-22,0,0,0
120,Namibia,2020-01-22,0,0,0
121,Nepal,2020-01-22,0,0,0
122,Netherlands,2020-01-22,0,0,0
123,New Zealand,2020-01-22,0,0,0
...,...,...,...,...,...
22624,Gambia,2020-05-21,1,24,13
22625,Georgia,2020-05-21,12,721,485
22626,Germany,2020-05-21,8203,179021,158087
22617,Estonia,2020-05-21,64,1800,1488


### Issue 6:
#### Observe:
- Consistency: Some countries are referred to with varying names, for example 'US' and 'United Stats'. Other names are not valid.

#### Define:
- Clean names of all dataframes by extracting country names with regex expressions and renaming countries.

#### Code

In [52]:
# Rename country values inplace
df_covid['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)
df_JHU_Countries_clean['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)

In [53]:
# Extract country name from column 'Entity' via regex
df_OWID_Testing_clean['country'] = df_OWID_Testing_clean.Entity.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
#df_OWID_Testing_clean.drop(['Entity'], axis=1, inplace=True)

In [54]:
# Extract country name from column 'Country or Area' via regex
df_UN_births_clean['country'] = df_UN_births_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

df_UN_deaths_clean['country'] = df_UN_deaths_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_UN_births_clean.drop(['Country or Area'], axis=1, inplace=True)
df_UN_deaths_clean.drop(['Country or Area'], axis=1, inplace=True)

In [55]:
# Extract country name from column 'Entity' via regex
df_WIKI_ICU_clean['country'] = df_WIKI_ICU_clean.countryname.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_WIKI_ICU_clean.drop(['countryname'], axis=1, inplace=True)

In [56]:
# Rename country values inplace
df_UN_births_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands',
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)

df_UN_deaths_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands', 
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)


df_OWID_Countries_clean['country'].replace({'Macedonia': 'North Macedonia'}, inplace=True)


df_ESTAT_census_clean['country'].replace({'Germany (until 1990 former territory of the FRG)': 
                                          'Germany'}, inplace=True)

In [57]:
# Create array with countries of interest
countries =['Afghanistan',
             'Albania',
             'Algeria',
             'American Samoa',
             'Andorra',
             'Angola',
             'Anguilla',
             'Antigua and Barbuda',
             'Argentina',
             'Armenia',
             'Aruba',
             'Australia',
             'Austria',
             'Azerbaijan',
             'Bahamas',
             'Bahrain',
             'Bangladesh',
             'Barbados',
             'Belarus',
             'Belgium',
             'Belize',
             'Benin',
             'Bermuda',
             'Bhutan',
             'Bolivia',
             'Bonaire Sint Eustatius and Saba',
             'Bosnia and Herzegovina',
             'Botswana',
             'Brazil',
             'British Virgin Islands',
             'Brunei',
             'Bulgaria',
             'Burkina Faso',
             'Burma',
             'Burundi',
             'Cabo Verde',
             'Cambodia',
             'Cameroon',
             'Canada',
             'Cape Verde',
             'Cayman Islands',
             'Central African Republic',
             'Chad',
             'Chile',
             'China',
             'Colombia',
             'Comoros',
             'Congo',
             'Cook Islands',
             'Costa Rica',
             "Cote d'Ivoire",
             'Croatia',
             'Cuba',
             'Cura',
             'Curacao',
             'Cyprus',
             'Czech Republic',
             'Czechia',
             'Denmark',
             'Diamond Princess',
             'Djibouti',
             'Dominica',
             'Dominican Republic',
             'Ecuador',
             'Egypt',
             'El Salvador',
             'Equatorial Guinea',
             'Eritrea',
             'Estonia',
             'Eswatini',
             'Ethiopia',
             'Faeroe Islands',
             'Falkland Islands',
             'Faroe Islands',
             'Fiji',
             'Finland',
             'France',
             'French Polynesia',
             'Gabon',
             'Gambia',
             'Georgia',
             'Germany',
             'Ghana',
             'Gibraltar',
             'Greece',
             'Greenland',
             'Grenada',
             'Guam',
             'Guatemala',
             'Guernsey',
             'Guinea',
             'Guinea-Bissau',
             'Guyana',
             'Haiti',
             'Holy See',
             'Honduras',
             'Hong Kong',
             'Hungary',
             'Iceland',
             'India',
             'Indonesia',
             'Iran',
             'Iraq',
             'Ireland',
             'Islands',
             'Isle',
             'Isle of Man',
             'Israel',
             'Italy',
             'Jamaica',
             'Japan',
             'Jersey',
             'Jordan',
             'Kazakhstan',
             'Kenya',
             'Korea, South',
             'Kosovo',
             'Kuwait',
             'Kyrgyzstan',
             'Laos',
             'Latvia',
             'Lebanon',
             'Lesotho',
             'Liberia',
             'Libya',
             'Liechtenstein',
             'Lithuania',
             'Luxembourg',
             'MS Zaandam',
             'Madagascar',
             'Malawi',
             'Malaysia',
             'Maldives',
             'Mali',
             'Malta',
             'Mauritania',
             'Mauritius',
             'Mexico',
             'Moldova',
             'Monaco',
             'Mongolia',
             'Montenegro',
             'Montserrat',
             'Morocco',
             'Mozambique',
             'Myanmar',
             'Namibia',
             'Nepal',
             'Netherlands',
             'New Caledonia',
             'New Zealand',
             'Nicaragua',
             'Niger',
             'Nigeria',
             'North Macedonia',
             'Northern Mariana Islands',
             'Norway',
             'Oman',
             'Pakistan',
             'Palau',
             'Palestine',
             'Panama',
             'Papua New Guinea',
             'Paraguay',
             'Peru',
             'Philippines',
             'Poland',
             'Portugal',
             'Puerto Rico',
             'Qatar',
             'Republic',
             'Romania',
             'Russia',
             'Rwanda',
             'Saint Helena',
             'Saint Kitts and Nevis',
             'Saint Lucia',
             'Saint Vincent',
             'Saint Vincent and the Grenadines',
             'San Marino',
             'Sao Tome and Principe',
             'Saudi Arabia',
             'Senegal',
             'Serbia',
             'Seychelles',
             'Sierra Leone',
             'Singapore',
             'Sint Maarten (Dutch part)',
             'Slovakia',
             'Slovenia',
             'Somalia',
             'South Africa',
             'South Korea',
             'South Sudan',
             'Spain',
             'Sri Lanka',
             'Sudan',
             'Suriname',
             'Swaziland',
             'Sweden',
             'Switzerland',
             'Syria',
             'Taiwan',
             'Tajikistan',
             'Tanzania',
             'Thailand',
             'Timor',
             'Timor-Leste',
             'Togo',
             'Trinidad and Tobago',
             'Tunisia',
             'Turkey',
             'Turks and Caicos Islands',
             'Uganda',
             'Ukraine',
             'United Arab Emirates',
             'United Kingdom',
             'United States',
             'United States Virgin Islands',
             'Uruguay',
             'Uzbekistan',
             'Vatican',
             'Venezuela',
             'Vietnam',
             'West Bank and Gaza',
             'Western Sahara',
             'World',
             'Yemen',
             'Zambia',
             'Zimbabwe']

In [58]:
# Only keep Countries of interes
df_covid = df_covid[df_covid['country'].isin(countries)]
df_JHU_Countries_clean = df_JHU_Countries_clean[df_JHU_Countries_clean['country'].isin(countries)]
df_OWID_Covid_clean = df_OWID_Covid_clean[df_OWID_Covid_clean['country'].isin(countries)]
df_OWID_Testing_clean = df_OWID_Testing_clean[df_OWID_Testing_clean['country'].isin(countries)]
df_OWID_Countries_clean = df_OWID_Countries_clean[df_OWID_Countries_clean['country'].isin(countries)]
df_WIKI_ICU_clean = df_WIKI_ICU_clean[df_WIKI_ICU_clean['country'].isin(countries)]
df_UN_births_clean = df_UN_births_clean[df_UN_births_clean['country'].isin(countries)]
df_UN_deaths_clean = df_UN_deaths_clean[df_UN_deaths_clean['country'].isin(countries)]
df_ESTAT_census_clean = df_ESTAT_census_clean[df_ESTAT_census_clean['country'].isin(countries)]

#### Test:

In [59]:
# Visually check countries of all dataframes

print(list(pd.Series(df_covid['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_JHU_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Covid_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Testing_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_WIKI_ICU_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_births_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_deaths_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_ESTAT_census_clean['country'].unique()).sort_values()))
print('\n\n')

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', '

In [60]:
# Check for countries which are referred to by different names in different dataframes
c_df_covid = df_covid['country'].unique()
c_df_JHU_Countries_clean = df_JHU_Countries_clean['country'].unique()
c_df_OWID_Covid_clean = df_OWID_Covid_clean['country'].unique()
c_df_OWID_Testing_clean = df_OWID_Testing_clean['country'].unique()
c_df_OWID_Countries_clean = df_OWID_Countries_clean['country'].unique()
c_df_WIKI_ICU_clean = df_WIKI_ICU_clean['country'].unique()
c_df_UN_births_clean = df_UN_births_clean['country'].unique()
c_df_UN_deaths_clean = df_UN_deaths_clean['country'].unique()
c_df_ESTAT_census_clean = df_ESTAT_census_clean['country'].unique()

all_country_names = list(c_df_covid) + list(c_df_JHU_Countries_clean) + list(c_df_OWID_Covid_clean) + list(c_df_OWID_Testing_clean) + list(c_df_OWID_Countries_clean) + list(c_df_WIKI_ICU_clean) + list(c_df_UN_births_clean) + list(c_df_UN_deaths_clean) + list(c_df_ESTAT_census_clean)

# countries = pd.Series(all_country_names).sort_values()

# Create alphabetically sorted list of all countries in all dataframes
list(pd.Series(pd.Series(all_country_names).unique()).sort_values())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cura',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'E

In [61]:
assert len(pd.Series(all_country_names).unique()) == len(countries)

### Issue 7:
#### Observe:
- Tidiness: In the df_OWID_Testing dataframes there is data which is not of interest.

#### Define:
- Drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description' from df_OWID_Testing_clean

#### Code

In [62]:
list(df_OWID_Testing_clean)

['ISO code',
 'Entity',
 'date',
 'Source URL',
 'Source label',
 'Notes',
 'Number of observations',
 'Cumulative total',
 'Cumulative total per thousand',
 'Daily change in cumulative total',
 'Daily change in cumulative total per thousand',
 '7-day smoothed daily change',
 '7-day smoothed daily change per thousand',
 'General source label',
 'General source URL',
 'Short description',
 'Detailed description',
 'country']

In [63]:
# Drop columns
df_OWID_Testing_clean.drop(['Source URL',
                             'Source label',
                             'Notes',
                             'Number of observations',
                             'Daily change in cumulative total',
                             'Daily change in cumulative total per thousand',
                             '7-day smoothed daily change',
                             '7-day smoothed daily change per thousand',
                             'General source label',
                             'General source URL',
                             'Short description',
                             'Detailed description',
                           ], axis=1, inplace=True)

df_UN_births_clean.drop(['Area', 'Record Type', 'Reliability', 'Source Year', 'Value Footnotes'], axis=1, inplace=True)
df_UN_deaths_clean.drop(['Area', 'Record Type', 'Reliability', 'Source Year', 'Value Footnotes'], axis=1, inplace=True)

#### Test

In [64]:
df_UN_births_clean.head()


,Year,Month,Value,country
0,2017,Total,279.0,Islands
1,2017,January,25.0,Islands
2,2017,February,23.0,Islands
3,2017,March,19.0,Islands
4,2017,April,19.0,Islands


In [65]:
list(df_OWID_Testing_clean)

['ISO code',
 'Entity',
 'date',
 'Cumulative total',
 'Cumulative total per thousand',
 'country']

### Issue 8:
#### Observe:
- Datatype of variable population in df_OWID_Countries is not integer and there are variables not of interest for this project 'countriesAndTerritories' and 'population_year'

#### Define:
- Convert datatype of population to integer, drop columns 'countriesAndTerritories', 'population_year'

#### Code

In [66]:
# First replace na with zero to be able to convert to integer
df_OWID_Countries_clean = df_OWID_Countries_clean.fillna(0)
df_OWID_Countries_clean.population = df_OWID_Countries_clean.population.astype(int)

In [67]:
# Drop columns
df_OWID_Countries_clean.drop(['countriesAndTerritories', 'population_year'], axis=1, inplace=True)

#### Test

In [68]:
# Check columns
list(df_OWID_Countries_clean)

['country', 'continent', 'population']

In [69]:
# Check data types
df_OWID_Countries_clean.dtypes

country       object
continent     object
population     int64
dtype: object

### Issue 9:
#### Observe:
- Tidiniss: Colum 'Value' contains more than one variable and observations are spread over multiple rows. Each variable should be a column, each observation a row.

#### Define:
- Using groupby and unstack, create colums for each household size and store values for one year and one country in one row in df_ESTAT_census as described in [Stack Overflow](https://stackoverflow.com/questions/26255671/pandas-column-values-to-columns).

#### Code

In [70]:
# Rename columns
df_ESTAT_census_clean.rename(columns={'TIME': 'year', 'Value': 'percentage', 
                                            'N_PERSON': 'index'}, inplace=True)


In [71]:
# df_ESTAT_census_clean.pivot(index=df_ESTAT_census_clean.index, columns='sizeHousehold')['percentage']
df_ESTAT_census_clean = df_ESTAT_census_clean.reset_index().groupby([
    'year', 'country', 'index'])['percentage'].aggregate('first').unstack().reset_index()

In [72]:
# For 2019 too few values for countries available
df_ESTAT_census_clean = df_ESTAT_census_clean.query('year<2019')

In [73]:
# Replace ':' values with NaN
df_ESTAT_census_clean = df_ESTAT_census_clean.replace(to_replace=':', value =np.nan)

#### Test

In [74]:
# Check visually
df_ESTAT_census_clean

index,year,country,1 person,2 persons,3 persons,4 persons,5 persons,6 persons or more
0,2010,Austria,36.0,28.6,16.0,12.5,5.0,1.9
1,2010,Belgium,34.5,31.7,15.0,11.2,5.5,2.1
2,2010,Bulgaria,19.5,27.8,20.4,19.3,7.3,5.7
3,2010,Croatia,24.6,25.6,18.8,17.3,7.7,6.0
4,2010,Cyprus,20.8,30.2,18.3,16.9,7.8,6.0
...,...,...,...,...,...,...,...,...
310,2018,Spain,25.5,30.4,20.9,17.7,4.1,1.4
311,2018,Sweden,42.5,31.2,11.2,10.8,3.1,1.1
312,2018,Switzerland,36.8,32.8,12.6,12.5,4.0,1.3
313,2018,Turkey,NaN,NaN,NaN,NaN,NaN,NaN


### Issue 10:
#### Observe:
- Accuracy: Not all date columns have datatype datetime and float values are string.

#### Define:
- Convert columns to correct datatype.

#### Code

In [75]:
df_OWID_Testing_clean.date=pd.to_datetime(df_OWID_Testing_clean.date)


df_ESTAT_census_clean['1 person'] = df_ESTAT_census_clean['1 person'].astype(float)
df_ESTAT_census_clean['2 persons'] = df_ESTAT_census_clean['2 persons'].astype(float)
df_ESTAT_census_clean['3 persons'] = df_ESTAT_census_clean['3 persons'].astype(float)
df_ESTAT_census_clean['4 persons'] = df_ESTAT_census_clean['4 persons'].astype(float)
df_ESTAT_census_clean['5 persons'] = df_ESTAT_census_clean['5 persons'].astype(float)
df_ESTAT_census_clean['6 persons or more'] = df_ESTAT_census_clean['6 persons or more'].astype(float)


df_WIKI_ICU_clean['ICU-CCB_beds_per_1000_people'] = df_WIKI_ICU_clean['ICU-CCB_beds_per_1000_people'].astype(float)
df_WIKI_ICU_clean['ventilators'] = df_WIKI_ICU_clean['ventilators'].astype(float)

# First replace na with zero to be able to convert to integer
df_OWID_Covid_clean.population = df_OWID_Covid_clean.population.fillna(0)
df_OWID_Covid_clean.population = df_OWID_Covid_clean.population.astype(int)

#### Test

In [76]:
df_OWID_Testing_clean.dtypes

ISO code                                 object
Entity                                   object
date                             datetime64[ns]
Cumulative total                          int64
Cumulative total per thousand           float64
country                                  object
dtype: object

In [77]:
df_ESTAT_census_clean.dtypes

index
year                   int64
country               object
1 person             float64
2 persons            float64
3 persons            float64
4 persons            float64
5 persons            float64
6 persons or more    float64
dtype: object

In [78]:
df_WIKI_ICU_clean.dtypes

continent                         object
hospital_beds_per_1000_people    float64
occupancy                        float64
ICU-CCB_beds_per_1000_people     float64
ventilators                      float64
country                           object
dtype: object

In [79]:
df_OWID_Covid_clean.dtypes

iso_code                            object
country                             object
date                                object
new_tests_smoothed                 float64
new_tests_smoothed_per_thousand    float64
stringency_index                   float64
population                           int64
population_density                 float64
median_age                         float64
aged_65_older                      float64
aged_70_older                      float64
gdp_per_capita                     float64
diabetes_prevalence                float64
female_smokers                     float64
male_smokers                       float64
hospital_beds_per_100k             float64
dtype: object

### Issue 11:
#### Observe:
-  merge df_OWID_Countries with df_country

#### Define:
- Using merge

#### Code

In [80]:
# covid cases timeline
iso_code = df_OWID_Covid_clean[['country', 'iso_code']].copy()
iso_code.drop_duplicates(inplace=True)
df_covidCases_Timeline = pd.merge(df_covid, iso_code, on=['country'])

In [81]:
# covid tests snapshot
df_covidTests_Snapshot = df_OWID_Testing_clean.copy()

In [82]:
# country metrics snapshot
df_x = df_OWID_Covid_clean.copy()
df_x.drop(['population'], axis=1, inplace=True)

# Only keep data of one date as snapshot
dateSnapshot = pd.to_datetime('2020-05-17')
df_x.date=pd.to_datetime(df_x.date)
df_x = df_x.query('date == @dateSnapshot')

df_countryMetrics_Snapshot = pd.merge(df_OWID_Countries_clean, df_x, on=['country'])

In [83]:
df_x.query('country == "Germany"')

,iso_code,country,date,new_tests_smoothed,new_tests_smoothed_per_thousand,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_100k
4506,DEU,Germany,2020-05-17,NaN,NaN,68.26,237.016,46.6,21.453,15.957,45229.245,8.31,28.2,33.1,8.0


In [84]:
# household timeline
df_householdsEU_Timeline = df_ESTAT_census_clean.copy()

In [85]:
# hospital snapshot
df_hospitals_Snapshot = df_WIKI_ICU_clean.copy()

In [86]:
# population delta timeline
df_UN_births_clean.rename(columns={'Value': 'births'}, inplace=True)
df_UN_deaths_clean.rename(columns={'Value': 'deaths'}, inplace=True)

df_populationDelta_Timeline = pd.merge(df_UN_births_clean, df_UN_deaths_clean, on=['country', 'Year', 'Month'])

#### Test

In [87]:
df_covidCases_Timeline.head()

,country,date,fatal,confirmed,recovered,iso_code
0,Afghanistan,2020-01-22,0,0,0,AFG
1,Afghanistan,2020-01-23,0,0,0,AFG
2,Afghanistan,2020-01-24,0,0,0,AFG
3,Afghanistan,2020-01-25,0,0,0,AFG
4,Afghanistan,2020-01-26,0,0,0,AFG


In [88]:
df_covidTests_Snapshot.head()

,ISO code,Entity,date,Cumulative total,Cumulative total per thousand,country
0,ARG,Argentina - tests performed,2020-05-20,112370,2.486,Argentina
1,AUS,Australia - tests performed,2020-05-20,1111567,43.591,Australia
2,AUT,Austria - tests performed,2020-05-20,379592,42.147,Austria
3,BHR,Bahrain - units unclear,2020-05-20,255633,150.232,Bahrain
4,BGD,Bangladesh - samples tested,2020-05-19,193645,1.176,Bangladesh


In [89]:
df_countryMetrics_Snapshot.head()

,country,continent,population,iso_code,date,new_tests_smoothed,new_tests_smoothed_per_thousand,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_100k
0,Afghanistan,Asia,38928341,AFG,2020-05-17,NaN,NaN,76.33,54.422,18.6,2.581,1.337,1803.987,9.59,NaN,NaN,0.50
1,Albania,Europe,2877800,ALB,2020-05-17,NaN,NaN,NaN,104.871,38.0,13.188,8.643,11803.431,10.08,7.1,51.2,2.89
2,Algeria,Africa,43851043,DZA,2020-05-17,NaN,NaN,NaN,17.348,29.1,6.211,3.857,13913.839,6.73,0.7,30.4,1.90
3,Andorra,Europe,77265,AND,2020-05-17,NaN,NaN,56.34,163.755,NaN,NaN,NaN,NaN,7.97,29.0,37.8,NaN
4,Angola,Africa,32866268,AGO,2020-05-17,NaN,NaN,NaN,23.890,16.8,2.405,1.362,5819.495,3.94,NaN,NaN,NaN


In [90]:
df_householdsEU_Timeline.head()

index,year,country,1 person,2 persons,3 persons,4 persons,5 persons,6 persons or more
0,2010,Austria,36.0,28.6,16.0,12.5,5.0,1.9
1,2010,Belgium,34.5,31.7,15.0,11.2,5.5,2.1
2,2010,Bulgaria,19.5,27.8,20.4,19.3,7.3,5.7
3,2010,Croatia,24.6,25.6,18.8,17.3,7.7,6.0
4,2010,Cyprus,20.8,30.2,18.3,16.9,7.8,6.0


In [91]:
df_hospitals_Snapshot.head()

,continent,hospital_beds_per_1000_people,occupancy,ICU-CCB_beds_per_1000_people,ventilators,country
0,Asia,13.05,75.5,7.3,32586.0,Japan
1,Asia,12.27,NaN,10.6,9795.0,South Korea
2,Europe,8.05,NaN,8.3,40000.0,Russia
3,Europe,8.00,79.8,33.9,40000.0,Germany
4,Europe,7.37,73.8,21.8,2500.0,Austria


In [92]:
df_populationDelta_Timeline.head()

,Year,Month,births,country,deaths
0,2017,Total,279.0,Islands,235.0
1,2017,January,25.0,Islands,17.0
2,2017,February,23.0,Islands,7.0
3,2017,March,19.0,Islands,24.0
4,2017,April,19.0,Islands,19.0


<a id='store'></a>
## 5. Store clean data

In [93]:
folder_name = 'outputData'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)   

In [94]:
# Store cleaned dataset to csv
df_covidCases_Timeline.to_csv('outputData/df_covidCases_Timeline.csv', encoding='utf-8')
df_covidTests_Snapshot.to_csv('outputData/df_covidTests_Snapshot.csv', encoding='utf-8')
df_countryMetrics_Snapshot.to_csv('outputData/df_countryMetrics_Snapshot.csv', encoding='utf-8')
df_householdsEU_Timeline.to_csv('outputData/df_householdsEU_Timeline.csv', encoding='utf-8')
df_hospitals_Snapshot.to_csv('outputData/df_hospitals_Snapshot.csv', encoding='utf-8')
df_populationDelta_Timeline.to_csv('outputData/df_populationDelta_Timeline.csv', encoding='utf-8')